In [1]:
import pandas as pd
import os
#import sqlite3
import psycopg2
import csv

In [2]:
file_names = []
for (dirpath, dirnames, filenames) in os.walk('.'):
    file_names.extend(filenames)
    break
    
file_names = list(filter(lambda x:'csv' in x, filenames))
file_names

['subjects.csv',
 'subject_memberships.csv',
 'rooms.csv',
 'schedules.csv',
 'sections.csv',
 'courses.csv',
 'course_offerings.csv',
 'instructors.csv',
 'teachings.csv',
 'grade_distributions.csv']

In [3]:
tables = {}
for table in file_names:
    print(table)
    with open(table) as f:
        tables[table[:-4]] = pd.read_csv(f)
        
len(tables)

subjects.csv
subject_memberships.csv
rooms.csv
schedules.csv
sections.csv
courses.csv
course_offerings.csv
instructors.csv
teachings.csv
grade_distributions.csv


10

In [4]:
for table in tables.keys():
    print(table, ' : ', list(tables[table].columns))

subjects  :  ['code', 'name', 'abbreviation']
subject_memberships  :  ['subject_code', 'course_offering_uuid']
rooms  :  ['uuid', 'facility_code', 'room_code']
schedules  :  ['uuid', 'start_time', 'end_time', 'mon', 'tues', 'wed', 'thurs', 'fri', 'sat', 'sun']
sections  :  ['uuid', 'course_offering_uuid', 'section_type', 'number', 'room_uuid', 'schedule_uuid']
courses  :  ['uuid', 'name', 'number']
course_offerings  :  ['uuid', 'course_uuid', 'term_code', 'name']
instructors  :  ['id', 'name']
teachings  :  ['instructor_id', 'section_uuid']
grade_distributions  :  ['course_offering_uuid', 'section_number', 'a_count', 'ab_count', 'b_count', 'bc_count', 'c_count', 'd_count', 'f_count', 's_count', 'u_count', 'cr_count', 'n_count', 'p_count', 'i_count', 'nw_count', 'nr_count', 'other_count']


In [15]:
tables['course_offerings']['course_uuid'].value_counts()

4d6a5df7-3427-3650-b946-ce5c616ff530    22
8ca4cb82-0242-32e6-8206-b1f99ae17e84    22
4590d10f-d7d6-333c-8b79-44ce35b33051    22
1138ead6-bacb-302d-931a-bd2b4415c06c    22
26093156-2a6b-34ff-b4fd-b7e47b18fbc7    22
341c1321-f72a-3fce-8dd9-2f99884fef5f    22
13da2ed6-f4cc-3d92-bab3-cb3a02c1bbfc    22
836ed4e1-a900-3f19-8d5b-ce8f1e4fa8bc    22
3b2fedd8-0705-3747-8276-8e098af899fe    22
2997300e-efa3-3463-aedc-375f47959144    22
e4884c26-a42a-3f14-8f08-bb325d9c0358    22
0f714438-313d-3687-a92c-f12d65427c60    22
0ce08969-507a-39b7-888f-4c253cebced9    22
53225a2a-88ce-3ef5-b469-e92bdbc5a90b    22
568800f7-0758-3a7f-8c83-3d29ebd58a44    22
d193425d-2bf1-33f4-9e3e-978ee8d07e39    22
8082de80-6263-347e-b7c0-7ccd3677d8b6    22
143bacc0-719d-3522-a649-958e834bc456    22
2a44f41b-75f0-315b-ab48-1570f161dd49    22
d69e3a14-4e6b-332d-96c6-911d3dfc69a8    22
ed57ad63-a33a-3d18-aa60-94a39594822b    22
bffbcd70-d12d-31eb-b0d7-a3bda7f425f8    22
ef810a32-9440-384b-bd01-6262253018ce    22
7c604c54-f6

In [ ]:
df = tables['rooms']
df['facility_code'].value_counts();

In [12]:
table = file_names[-7]
num_cols = len(tables[table[:-4]].columns)
cols = "%s," * num_cols
query = "INSERT INTO " + table[:-4] + " VALUES (" + cols[:-1] + ")"
query

'INSERT INTO schedules VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

In [ ]:
# row = ['5716145', "REMY FERNÁNDEZ-O'BRIEN"]
# for table in file_names:
#     print(f"{table[:-4]} has {len(tables[table[:-4]].columns)} columns")
#     query = f"""INSERT INTO {table[:-4]} VALUES {row}"""
#     print(query)

In [ ]:
# file_names[-3]
# tables['instructors'].head()

In [14]:
conn = psycopg2.connect("dbname=uw-grades")
cur = conn.cursor()

for table in file_names[-3:-2]:
    with open(table, 'r') as f:
        num_cols = len(tables[table[:-4]].columns)
        cols = "%s," * num_cols
        reader = csv.reader(f)
        next(reader)  # Skip the header row.
        for row in reader:
            query = "INSERT INTO " + table[:-4] + " VALUES (" + cols[:-1] + ")"
            cur.execute(query, row)
            
    conn.commit()

In [ ]:
conn = sqlite3.connect('database.sqlite3')
conn.set_trace_callback(print)
#conn.tables
c = conn.cursor()
test_table = 'teachings'
query = """SELECT sql FROM sql_master WHERE name = '""" + test_table + "';"
c.execute(query)
for _ in range(100):
    print(next(c))